In [1]:
import cudaq
import numpy as np
import random
from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*')
cudaq.set_target('nvidia')

@cudaq.kernel
def h2_molecule_simulation(weight_vector: list[float]):
    # 创建包含6个量子比特的量子寄存器
    q = cudaq.qvector(6)
    
    # 应用相应的量子门
    # atom_1: q[0], q[1]
    # atom_2: q[2], q[3]
    # bond_2_1: q[4], q[5]
    
    ry(np.pi * weight_vector[0], q[0])
    x(q[1])
    ry(np.pi * weight_vector[2], q[2])
    ry(np.pi * weight_vector[4], q[3])
    
    x.ctrl(q[0], q[1])
    ry.ctrl(np.pi * weight_vector[3], q[1], q[2])
    x.ctrl(q[2], q[3])
    ry.ctrl(np.pi * weight_vector[1], q[0], q[1])
    x.ctrl(q[1], q[2])
    ry.ctrl(np.pi * weight_vector[5], q[2], q[3])
    
    # 测量前4个量子比特
    m0 = mz(q[0])
    m1 = mz(q[1])
    m2 = mz(q[2])
    m3 = mz(q[3])
    
    # 在CUDA-Q中实现条件操作
    if (m2 == 0) and (m3 == 0):
        # 如果测量结果为0，跳过操作
        pass
    else:
        # 否则，执行下面的操作
        ry(np.pi * weight_vector[6], q[4])
        x(q[5])
        x.ctrl(q[4], q[5])
        ry.ctrl(np.pi * weight_vector[7], q[4], q[5])
    
    # 最后测量所有比特（将自动应用到尚未测量的比特上）
    mz(q)


weight_vector = np.array([random.random() for _ in range(8)])

# 运行量子电路并获取结果
result = cudaq.sample(h2_molecule_simulation, weight_vector)

# 打印结果
result = dict(result.items())
print(result)

{'011111': 1, '100101': 9, '111011': 2, '111101': 4, '010110': 53, '100111': 22, '010000': 169, '100110': 1, '010101': 156, '111110': 1, '111111': 13, '100000': 106, '010111': 463}


In [20]:
import sys
sys.path.append("../")
from qmg.utils import MoleculeQuantumStateGenerator

mg = MoleculeQuantumStateGenerator(heavy_atom_size=2)

for key, count in result.items():
    print(mg.QuantumStateToSmiles(key), count)

O 1
None 3
None 1
None 7
CC 3
None 4
C#N 14
C=O 5
C=N 80
N#N 44
CN 1
N=N 55
C 315
C#N 46
C=C 162
N 105
N=O 7
None 1
O=O 1
C#C 111
N=O 14
C=N 20


In [2]:
def create_molecule_circuit(num_heavy_atom=9):
    num_parameters = int(8 + 3*(num_heavy_atom - 2) * (num_heavy_atom + 3) / 2)
    weight_vector = np.array([random.random() for _ in range(num_parameters)])
    
    # 计算所需的总量子比特数
    # 2个用于atom_1, 2个用于atom_i, 每个键(bond)2个
    total_qubits = 2 + 2 + 2 * (num_heavy_atom - 1)
    
    @cudaq.kernel
    def molecule_simulation(weights: list[float]):
        q = cudaq.qvector(total_qubits)
        
        # atom_1: q[0:2]
        # atom_i: q[2:4]
        # bond_k: q[4+2*(k-1):6+2*(k-1)]
        
        ry(np.pi * weights[0], q[0])
        x(q[1])
        ry(np.pi * weights[2], q[2])
        ry(np.pi * weights[4], q[3])
        
        x.ctrl(q[0], q[1])
        ry.ctrl(np.pi * weights[3], q[1], q[2])
        x.ctrl(q[2], q[3])
        ry.ctrl(np.pi * weights[1], q[0], q[1])
        x.ctrl(q[1], q[2])
        ry.ctrl(np.pi * weights[5], q[2], q[3])
        
        atom_1_m0 = mz(q[0])
        atom_1_m1 = mz(q[1])
        atom_2_m0 = mz(q[2])
        atom_2_m1 = mz(q[3])
        
        if atom_2_m0 != 0:  
            ry(np.pi * weights[6], q[4])
            x(q[5])
            x.ctrl(q[4], q[5])
            ry.ctrl(np.pi * weights[7], q[4], q[5])
        
        bond_2_1_m0 = mz(q[4])
        bond_2_1_m1 = mz(q[5])
        
        used_part = 8
        
        # 第三个原子和递归部分
        for heavy_atom_idx in range(3, num_heavy_atom+1):
            # 重置量子比特逻辑（这在CUDA-Q中需要特殊处理）
            # 在CUDA-Q中没有直接的重置操作，我们用条件X门模拟
            
            # 重置atom_i如果测量为1
            if atom_2_m0:
                x(q[2])  # 如果为1，翻转回0
            if atom_2_m1:
                x(q[3])  # 如果为1，翻转回0
            
            # 重置之前的bond量子比特
            for k in range(1, heavy_atom_idx-1):
                bond_idx_start = 4 + 2*(k-1)
                bond_m0 = mz(q[bond_idx_start])
                bond_m1 = mz(q[bond_idx_start+1])
                
                if bond_m0:
                    x(q[bond_idx_start])
                if bond_m1:
                    x(q[bond_idx_start+1])
            
            # 条件操作 - 基于前一个原子的测量
            if atom_2_m0 != 0:  # 非0情况
                ry(np.pi * weights[used_part], q[2])
                ry(np.pi * weights[used_part+1], q[3])
                ry.ctrl(np.pi * weights[used_part+2], q[2], q[3])
            
            # 测量atom_i作为新的当前原子
            atom_3_m0 = mz(q[2])
            atom_3_m1 = mz(q[3])
            
            # 条件操作 - 基于当前原子的测量
            if atom_3_m0 != 0:  # 非0情况
                for i in range(heavy_atom_idx-1):
                    bond_idx_start = 4 + 2*i
                    ry(np.pi * weights[used_part+2+3*i+1], q[bond_idx_start+1])
                    ry.ctrl(np.pi * weights[used_part+2+3*i+2], q[bond_idx_start+1], q[bond_idx_start])
                    ry.ctrl(np.pi * weights[used_part+2+3*i+3], q[bond_idx_start], q[bond_idx_start+1])
            
            # 测量所有相关的bond量子比特
            for k in range(1, heavy_atom_idx):
                bond_idx_start = 4 + 2*(k-1)
                mz(q[bond_idx_start])
                mz(q[bond_idx_start+1])
            
            # 更新使用的参数索引
            used_part += 3*heavy_atom_idx
            
            # 更新atom_2_m的值以便下一次迭代
            atom_2_m0 = atom_3_m0
            atom_2_m1 = atom_3_m1
        
        # 最终测量剩余的量子比特(如果有的话)
        mz(q)
    
    # 运行模拟
    result = cudaq.sample(molecule_simulation, weight_vector, shots_count=10000)
    return result



In [3]:
num_heavy_atom = 9
result = create_molecule_circuit(num_heavy_atom)
print(f"分子电路模拟结果 (重原子数: {num_heavy_atom}):")
result = dict(result.items())
print(result)

分子电路模拟结果 (重原子数: 9):
{'01110011000100000000': 1, '11101111000000000000': 1, '10101101000000010000': 1, '10110100001100000100': 1, '10110001000011010100': 1, '10111101000000010000': 2, '01101101000000000000': 1, '10100000000000000100': 1, '10101111001100000100': 1, '10110111000000000101': 1, '10100111000000010001': 1, '10110011001111000100': 1, '10101111000000000001': 1, '11100010000000000101': 1, '10100000000100000000': 1, '10101001001100000100': 1, '11110101000000000001': 1, '01100010000100010000': 1, '11110000000000000100': 1, '11100010000000000100': 1, '11110101000100000001': 1, '10101010000000000001': 1, '10110010000100000001': 2, '01110101000000000100': 1, '11100010000100000000': 1, '10110010000100000100': 1, '10110110000100000001': 1, '10100001000101000100': 1, '10110010000010010001': 1, '11110111000100000000': 1, '01101101000000000100': 1, '11110110000100000100': 1, '10100101000100000010': 1, '11101101000000000101': 1, '01110101000000010000': 1, '10110001000001000100': 1, '111000

In [24]:
mg = MoleculeQuantumStateGenerator(heavy_atom_size=num_heavy_atom)

for key, count in result.items():
    print(mg.QuantumStateToSmiles(key), count)

C.C.C=O 1
C.C.C.N.N.O 1
C.O.OO 1
None 1
C.CN.O.O 1
None 1
C.CN.O 1
C.CN.O.O 1
C.C.N.O 1
C.C.C.N.O 1
C.C.C.C=O.O 1
C.C.CN.O.O 1
C.C.NO 1
C.C.C.O.O.O 1
C.CO.O.O 1
C.C.CN.O 1
C.C.N.O.O 1
C.O.O.O.O 1
C.C.C.O.O 1
C.C.C.CO 1
C.C#N.O.O 1
C.C.N.O.O 1
C.O.O.O 1
C.C.C.C.C.N 2
None 1
C.CO.N.O 1
C.C.C.C#N 1
C.C.CN.N 1
C.C.C.N.O 2
C.C.C#N 1
C.C.CO.N 1
C.C.C.C#N.O 2
None 3
C.C.C.NO 2
C.C.CO.N 1
C.C.C.C#N.O 1
C.C.C.CN.O 2
C.C#N.O 1
C.C.O.O 1
None 1
C.C.NO 1
C.C.C=N.O.O 1
C.C.C.C.C=N 1
None 2
C.C=O.O.O 1
C.N.O 3
C.C.C.CN.O 1
None 1
C.C.C.CO.O 1
C#N.O.O 1
C.C.CN.O 1
C.CO.O 5
C.OO 1
C.C.C.C.N 1
C.C.CO.O.O 2
C.C.C.O.O 1
C.NO.O.O 1
C.C.O.O.O 1
C.O.O.O 1
C.C.C.CN.O.O 1
C.C.C.C.OO 1
C.CO.O 1
C.C.C.C=O.O 1
C.O 42
C.C.C.N.O.O 1
C.C.C=N 6
C.C.CO.O 1
C.C.CO.O.O 1
None 1
C.C.C.C.NO 1
C.C.C.O 3
C.C.C=N.O 1
C.C.O.O=O 1
C.O.O 5
C.C.N=O 1
C.C.CO.O 8
C.C.OO 1
C.C.O.O=O 1
C.C.C.C.CN.O 2
C.C.C.CO 2
None 1
C 266
None 1
C.C.C.C.O 2
None 2
C.C.C.CN.O 2
C.C.C.C.CO 1
C.C.N.O.O.O 1
C.C.N.O 1
C.C.CN 1
C.C.C.CO 8
C.O.O.OO 1
C.

In [10]:
import cudaq
import numpy as np
import random

def create_molecule_circuit_with_mid_measurements(num_heavy_atom=9):
    # 计算所需参数数量
    num_parameters = int(8 + 3*(num_heavy_atom - 2) * (num_heavy_atom + 3) / 2)
    weight_vector = np.array([random.random() for _ in range(num_parameters)])
    
    # 计算原始电路所需的量子比特数
    original_qubits = 2 + 2 + 2 * (num_heavy_atom - 1)
    
    # 计算额外需要的量子比特数量来存储中间测量结果
    extra_qubits = 2 + 2 + 2 * (num_heavy_atom - 1)
    
    # 总量子比特数
    total_qubits = original_qubits + extra_qubits
    
    @cudaq.kernel
    def molecule_simulation_with_mapping(weights: list[float], orig_qubits: int):
        # 创建量子寄存器
        q = cudaq.qvector(total_qubits)
        
        # 第一部分: 前两个原子的操作
        ry(np.pi * weights[0], q[0])
        x(q[1])
        ry(np.pi * weights[2], q[2])
        ry(np.pi * weights[4], q[3])
        
        x.ctrl(q[0], q[1])
        ry.ctrl(np.pi * weights[3], q[1], q[2])
        x.ctrl(q[2], q[3])
        ry.ctrl(np.pi * weights[1], q[0], q[1])
        x.ctrl(q[1], q[2])
        ry.ctrl(np.pi * weights[5], q[2], q[3])
        
        # 模拟测量atom_1，将状态复制到辅助寄存器
        # 使用传入的orig_qubits参数而不是外部变量
        x.ctrl(q[0], q[orig_qubits])       # 如果q[0]=1，翻转辅助位
        x.ctrl(q[1], q[orig_qubits+1])     # 如果q[1]=1，翻转辅助位
        
        # 模拟测量atom_i (atom_2)
        x.ctrl(q[2], q[orig_qubits+2])     # 如果q[2]=1，翻转辅助位
        x.ctrl(q[3], q[orig_qubits+3])     # 如果q[3]=1，翻转辅助位
        
        # 获取测量结果（对于条件逻辑）
        atom_2_m0 = mz(q[2])
        
        # 条件操作 - 对应于if_test for atom_2_m
        if atom_2_m0 != 0:  # 非0情况，对应于else子句
            # 操作bond_1 (q[4:6])
            ry(np.pi * weights[6], q[4])
            x(q[5])
            x.ctrl(q[4], q[5])
            ry.ctrl(np.pi * weights[7], q[4], q[5])
        
        # 将bond_1的状态复制到相应的辅助量子比特
        x.ctrl(q[4], q[orig_qubits+4])
        x.ctrl(q[5], q[orig_qubits+5])
        
        # 获取bond_1测量结果（用于控制逻辑）
        bond_2_1_m0 = mz(q[4])
        bond_2_1_m1 = mz(q[5])
        
        used_part = 8
        
        # 第三个原子和递归部分
        for heavy_atom_idx in range(3, num_heavy_atom+1):
            # 重置操作
            if atom_2_m0:
                x(q[2])  # 如果为1，翻转回0
            
            atom_2_m1 = mz(q[3])
            if atom_2_m1:
                x(q[3])  # 如果为1，翻转回0
            
            # 重置之前的bond量子比特
            for k in range(1, heavy_atom_idx-1):
                bond_idx_start = 4 + 2*(k-1)
                bond_m0 = mz(q[bond_idx_start])
                bond_m1 = mz(q[bond_idx_start+1])
                
                if bond_m0:
                    x(q[bond_idx_start])
                if bond_m1:
                    x(q[bond_idx_start+1])
            
            # 条件操作 - 基于前一个原子的测量
            if atom_2_m0 != 0:  # 非0情况
                ry(np.pi * weights[used_part], q[2])
                ry(np.pi * weights[used_part+1], q[3])
                ry.ctrl(np.pi * weights[used_part+2], q[2], q[3])
            
            # 测量atom_i作为新的当前原子（用于控制流程）
            atom_3_m0 = mz(q[2])
            atom_3_m1 = mz(q[3])
            
            # 条件操作 - 基于当前原子的测量
            if atom_3_m0 != 0:  # 非0情况
                for i in range(heavy_atom_idx-1):
                    bond_idx_start = 4 + 2*i
                    ry(np.pi * weights[used_part+2+3*i+1], q[bond_idx_start+1])
                    ry.ctrl(np.pi * weights[used_part+2+3*i+2], q[bond_idx_start+1], q[bond_idx_start])
                    ry.ctrl(np.pi * weights[used_part+2+3*i+3], q[bond_idx_start], q[bond_idx_start+1])
                    
                    # 将bond状态复制到对应的辅助量子比特
                    aux_idx_start = orig_qubits + 4 + 2*i
                    x.ctrl(q[bond_idx_start], q[aux_idx_start])
                    x.ctrl(q[bond_idx_start+1], q[aux_idx_start+1])
            
            # 更新使用的参数索引
            used_part += 3*heavy_atom_idx
            
            # 更新atom_2_m的值以便下一次迭代
            atom_2_m0 = atom_3_m0
        
        # 最终测量所有量子比特
        mz(q)
    
    # 运行模拟 - 将original_qubits作为参数传入
    result = cudaq.sample(molecule_simulation_with_mapping, weight_vector, original_qubits, shots_count=10000)
    return result, original_qubits

# 使用示例
def analyze_results(num_heavy_atom=9):
    result, original_qubits = create_molecule_circuit_with_mid_measurements(num_heavy_atom)
    result = dict(result.items())
    print(result)
    # 分析结果，提取中间测量信息
    print(f"获取模拟结果（共{result.counts()}次测量）:")
    
    total_shots = result.count()
    
    for bitstring, count in result.count().items():
        # 分离原始量子比特和辅助量子比特
        original_bits = bitstring[:original_qubits]
        aux_bits = bitstring[original_qubits:]
        
        # 提取中间测量结果
        atom_1_result = aux_bits[0:2]
        atom_i_result = aux_bits[2:4]
        
        # 打印重要的中间测量结果样本（概率较高的）
        if count > total_shots * 0.01:  # 只显示概率>1%的结果
            print(f"结果: {bitstring}, 计数: {count}, 概率: {count/total_shots:.4f}")
            print(f"  - 原始比特: {original_bits}")
            print(f"  - 辅助比特: {aux_bits}")
            print(f"  - atom_1 中间测量: {atom_1_result}")
            print(f"  - atom_i 中间测量: {atom_i_result}")
            
            # 提取bond的中间测量结果
            for k in range(1, num_heavy_atom-1):
                bond_start = 4 + 2*(k-1)
                bond_result = aux_bits[bond_start:bond_start+2]
                print(f"  - bond_{k+1}_{k} 中间测量: {bond_result}")
            
            print()
    
    return result

# 主程序
if __name__ == "__main__":
    # 可以指定不同的重原子数量
    num_heavy_atom = 3  # 从较小的值开始测试
    analyze_results(num_heavy_atom)

{'0111000001111000': 1, '1011010010100000': 1, '0111011101110011': 1, '0111001101110111': 2, '1011010110110001': 1, '0111000101100101': 1, '1001000010110100': 2, '1110000111110101': 1, '1100000011010000': 2, '0100000001111000': 6, '0111000101111001': 1, '0101000001111000': 5, '1000000010110100': 1, '0111010101110001': 265, '0100000001000000': 4058, '1000000010010000': 14, '0111000101110101': 856, '0111001001110110': 987, '0111010001110000': 36, '0110000101110101': 8, '0111001001111110': 1, '0100000001010000': 1205, '0111011001110010': 302, '0101000001110100': 1085, '0100000001111100': 2, '0111101001111110': 4, '1100000011110100': 9, '0100000001110100': 898, '0111000001100100': 1, '0111010101111101': 2, '0110001001110110': 15, '0111100001111100': 1, '0111100101111101': 3, '0111011001111110': 5, '0110011001110010': 6, '1101000011110100': 18, '1111000111110101': 16, '0110000001110100': 2, '1011000110110101': 1, '1111010011110000': 1, '0111000001110100': 119, '1000000010000000': 7, '011001

AttributeError: 'dict' object has no attribute 'counts'